In [1]:
import json
import time

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [4]:
from tqdm import tqdm_notebook

In [5]:
base_url = "https://www.trustpilot.com"

In [6]:
def get_soup(url):
    return BeautifulSoup(requests.get(url).content, 'lxml')

In [7]:
data = {}

soup = get_soup(base_url + '/categories')
for category in soup.findAll('div', {'class' : 'subCategory___BRUDy'}):
    data[category['id']] = {}
    for sub_category in category.findAll('div', {'class' : 'subCategoryItem___3ksKz'}):
        '''Note : .find() will search and return the first occurence but .findAll() will return all occurences'''
        name = sub_category.find('span').text
        link = sub_category.find('a', {'class' : 'internal___1jK0Z typography___lxzyt weight-inherit___229vl navigation___2n5P8'})['href']
        data[category['id']][name] = link
        
''' data will have all sub-catogaries in JSON format 👍'''

' data will have all sub-catogaries in JSON format 👍'

In [8]:
def fetch_company_urls_form_page():
    a_list = driver.find_elements_by_xpath('//a[@class="internal___1jK0Z wrapper___26yB4"]')
    urls=[]
    for a in a_list:
        link = str(a.get_attribute('href'))
        if link.startswith('https://www.trustpilot.com/review/'):
            urls.append(link)
    dedup_urls = list(set(urls))
    return dedup_urls

In [12]:
def go_next_page():
    try:
        button = driver.find_element_by_xpath('//a[@class="paginationLinkNormalize___scOgG paginationLinkNext___1LQ14"]')
        print('next page found')
        return True, button
    except NoSuchElementException:
        print('No Next Page')
        return False, None

In [10]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument("--disable-extensions")

prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome('./driver/chromedriver', options=options)

timeout = 3

In [14]:
''' Fixing issue with website where /categories/gambling redirects to /categories/gambling_house '''

test = data['events_entertainment']
test['gambling_house'] = '/categories/gambling_house'
test['betting_agency'] = '/categories/betting_agency'
test['bookmaker'] = '/categories/bookmaker'
test['casino'] = '/categories/casino'
test['gambling_instructor'] = '/categories/gambling_instructor'
test['gambling_service'] = '/categories/gambling_service'
test['lottery_vendor'] = '/categories/lottery_vendor'
test['online_casino_or_bookmaker'] = '/categories/online_casino_or_bookmaker'
test['online_sports_betting'] = '/categories/online_sports_betting'
test.pop('Gambling', None)

'/categories/gambling'

In [29]:
company_urls = {}
wrong_redirects = []

In [34]:
for category in tqdm_notebook(data):
    for sub_category in tqdm_notebook(data[category], leave=False):
        
        ''' 0. Checking if sub-category already present. Useful in case of power faliure and re-running of this cell '''
        
        if ((sub_category in company_urls) and (company_urls[sub_category] != '')):
            print('Already Present : ', sub_category)
            continue
        else:
            company_urls[sub_category] = []
        
        ''' 1. Fetching URLs for each sub-category '''
        
        url = base_url + data[category][sub_category] + "?numberofreviews=0&timeperiod=0&status=all"
        print('URL = ',url)
        
        ''' 2. Printing Info '''
        
        print ("Current Sub-Category : ", category, '->', sub_category)
        driver.get(url)
        current_url = driver.current_url
        
        ''' 3. Checking for bad URL redirects '''
        
        if (url != current_url):
            print('Pre URL : ', url, '\nRedirected URL : ', current_url)
            wrong_redirects.append(sub_category)
            print('Bad URL redirect with : ', sub_category)
            break
            
        ''' 4. Check if current page has URLs for companies, else continue to next sub-category '''            
        try: 
            element_present = EC.presence_of_element_located(
                (By.CLASS_NAME, 'businessUnitCardsContainer___Qhix1'))
            WebDriverWait(driver, timeout).until(element_present)
        except:
            pass
        
        ''' 5. Add all review links in page to list '''   
        
        next_page = True
        count = 1
        
        while next_page:
            print('Current Page URL in Loop : ', driver.current_url)
            extracted_company_urls = fetch_company_urls_form_page()
            print("Scanned Page", count, "of ", sub_category, "URLs found : ", len(extracted_company_urls))
            company_urls[sub_category] += extracted_company_urls
            next_page, button = go_next_page()
            
            if next_page:
                count += 1
                next_url = base_url + data[category][sub_category] + "?numberofreviews=0&timeperiod=0&status=all" + f'&page={count}'
                driver.get(next_url)
                try: 
                    element_present = EC.presence_of_element_located(
                        (By.CLASS_NAME, 'businessUnitCardsContainer___Qhix1'))
                    WebDriverWait(driver, timeout).until(element_present)
                except:
                    print('No next page left')
                    pass

C:\Users\teeji\anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


C:\Users\teeji\anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Already Present :  Animal Health
Already Present :  Animal Parks & Zoo
Already Present :  Cats & Dogs
Already Present :  Horses & Riding
Already Present :  Pet Services
Already Present :  Pet Stores


Already Present :  Cosmetics & Makeup
Already Present :  Hair Care & Styling
Already Present :  Personal Care
Already Present :  Salons & Clinics
Already Present :  Tattoos & Piercings
Already Present :  Wellness & Spa
Already Present :  Yoga & Meditation


Already Present :  Administration & Services
Already Present :  Associations & Centers
Already Present :  HR & Recruiting
Already Present :  IT & Communication
Already Present :  Import & Export
Already Present :  Office Space & Supplies
Already Present :  Print & Graphic Design
Already Present :  Research & Development
URL =  https://www.trustpilot.com/categories/sales_marketing?numberofreviews=0&timeperiod=0&status=all
Current Sub-Category :  business_services -> Sales & Marketing
Current Page URL in Loop :  https://www.trustpilot.com/categories/sales_marketing?numberofreviews=0&timeperiod=0&status=all
Scanned Page 1 of  Sales & Marketing URLs found :  0
No Next Page
URL =  https://www.trustpilot.com/categories/shipping_logistics?numberofreviews=0&timeperiod=0&status=all
Current Sub-Category :  business_services -> Shipping & Logistics
Current Page URL in Loop :  https://www.trustpilot.com/categories/shipping_logistics?numberofreviews=0&timeperiod=0&status=all
Scanned Page 1 of  Ship

URL =  https://www.trustpilot.com/categories/architects_engineers?numberofreviews=0&timeperiod=0&status=all
Current Sub-Category :  construction_manufactoring -> Architects & Engineers
Current Page URL in Loop :  https://www.trustpilot.com/categories/architects_engineers?numberofreviews=0&timeperiod=0&status=all
Scanned Page 1 of  Architects & Engineers URLs found :  0
No Next Page
URL =  https://www.trustpilot.com/categories/building_materials?numberofreviews=0&timeperiod=0&status=all
Current Sub-Category :  construction_manufactoring -> Building Materials
Current Page URL in Loop :  https://www.trustpilot.com/categories/building_materials?numberofreviews=0&timeperiod=0&status=all
Scanned Page 1 of  Building Materials URLs found :  0
No Next Page
URL =  https://www.trustpilot.com/categories/chemicals_plastic?numberofreviews=0&timeperiod=0&status=all
Current Sub-Category :  construction_manufactoring -> Chemicals & Plastic
Current Page URL in Loop :  https://www.trustpilot.com/categori

KeyboardInterrupt: 

In [33]:
''' Checking current list of keys '''
company_urls.keys()


dict_keys(['Animal Health', 'Animal Parks & Zoo', 'Cats & Dogs', 'Horses & Riding', 'Pet Services', 'Pet Stores', 'Cosmetics & Makeup', 'Hair Care & Styling', 'Personal Care', 'Salons & Clinics', 'Tattoos & Piercings', 'Wellness & Spa', 'Yoga & Meditation', 'Administration & Services', 'Associations & Centers', 'HR & Recruiting', 'IT & Communication', 'Import & Export', 'Office Space & Supplies', 'Print & Graphic Design', 'Research & Development'])

In [15]:
''' Dropping failed keys and starting from point of corruption due to power faliure '''

incomplete_key = "Children's Entertainment"
company_urls.pop(key, None)

In [28]:
''' Verifying successful corrupt key removal '''

company_urls.keys()

dict_keys([])

In [18]:
data.keys()

dict_keys(['animals_pets', 'beauty_wellbeing', 'business_services', 'construction_manufactoring', 'education_training', 'electronics_technology', 'events_entertainment', 'food_beverages_tobacco', 'health_medical', 'hobbies_crafts', 'home_garden', 'home_services', 'legal_services_government', 'media_publishing', 'money_insurance', 'public_local_services', 'restaurants_bars', 'shopping_fashion', 'sports', 'travel_vacation', 'utilities', 'vehicles_transportation'])

In [19]:
''' Fixing Data Dictionary '''

company_urls.pop('Adult Entertainment', None)

corrupt_data_keys = ('animals_pets', 'beauty_wellbeing', 'business_services', 'construction_manufactoring', 'education_training', 'electronics_technology', 'events_entertainment')
for key in corrupt_data_keys:
    data.pop(key, None)

data.keys()

dict_keys(['food_beverages_tobacco', 'health_medical', 'hobbies_crafts', 'home_garden', 'home_services', 'legal_services_government', 'media_publishing', 'money_insurance', 'public_local_services', 'restaurants_bars', 'shopping_fashion', 'sports', 'travel_vacation', 'utilities', 'vehicles_transportation'])